In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager.findfont(font)

'/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

# 데이터 로드

In [2]:
import os
import pandas as pd

train_data_path ="data/train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [3]:
train_data.groupby('class').count()

,idx,conversation
class,,
갈취 대화,981,981
기타 괴롭힘 대화,1094,1094
직장 내 괴롭힘 대화,979,979
협박 대화,896,896


In [4]:
test_data = pd.read_json('data/test.json', orient='index')

test_data.head()

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
t_005,그러니까 빨리 말해. 선생님 제발 살려주십시오. 비밀번호 틀릴 때마다 손톱 하나씩...


In [5]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, t_000 to t_499
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    400 non-null    object
dtypes: object(1)
memory usage: 6.2+ KB


# 데이터 전처리

In [6]:
train_data[train_data.iloc[392]['conversation'] == train_data['conversation']]

,idx,class,conversation
29,29,기타 괴롭힘 대화,병신이 아이스크림 먹게 돼 있냐?\n난 먹으면 안 돼? 그만 좀 해.\n당연히 안 ...
392,392,기타 괴롭힘 대화,병신이 아이스크림 먹게 돼 있냐?\n난 먹으면 안 돼? 그만 좀 해.\n당연히 안 ...


In [7]:
train_data.duplicated('conversation').sum()

104

In [8]:
train_data = train_data.drop_duplicates(subset='conversation')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3846 entries, 0 to 3949
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           3846 non-null   int64 
 1   class         3846 non-null   object
 2   conversation  3846 non-null   object
dtypes: int64(1), object(2)
memory usage: 120.2+ KB


In [9]:
train_data.groupby('class').count()

,idx,conversation
class,,
갈취 대화,973,973
기타 괴롭힘 대화,1011,1011
직장 내 괴롭힘 대화,970,970
협박 대화,892,892


In [10]:
train_data.iloc[29,2]

'병신이 아이스크림 먹게 돼 있냐?\n난 먹으면 안 돼? 그만 좀 해.\n당연히 안 되지 너 공부 개못하는 찐따잖아.\n야 닭대가리가 말도 하네 동물농장 나가자.\n너네 학교폭력으로 신고하기 전에 그만 해.\n아이고 무서워라 살려주세요 으헝헝\n제발 좀 저리 가.\n나였으면 아이스크림 먹을 시간에 공부한다.\n멈춰!\n저러니까 쳐맞고 다니지'

In [11]:
import re

def preprocess_document(document):
    document = re.sub('[^ㄱ-ㅎ가-힣a-zA-Z0-9]', ' ', document)
    document = re.sub(' +', ' ', document)
    document = document.strip()
    
    return document


preprocess_document(train_data.iloc[29,2])

'병신이 아이스크림 먹게 돼 있냐 난 먹으면 안 돼 그만 좀 해 당연히 안 되지 너 공부 개못하는 찐따잖아 야 닭대가리가 말도 하네 동물농장 나가자 너네 학교폭력으로 신고하기 전에 그만 해 아이고 무서워라 살려주세요 으헝헝 제발 좀 저리 가 나였으면 아이스크림 먹을 시간에 공부한다 멈춰 저러니까 쳐맞고 다니지'

In [12]:
import sentencepiece as spm

corpus = train_data['conversation'].map(preprocess_document)

spm.SentencePieceTrainer.train(sentence_iterator=iter(corpus),
                               model_prefix='bpe',
                               vocab_size=5000,
                               max_sentence_length=9999,
                               model_type='bpe',
                              )

tokenizer = spm.SentencePieceProcessor()
tokenizer.load('bpe'+'.model')

tokenizer.encode_as_pieces(preprocess_document(train_data.iloc[29,2]))

sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input_format: 
  model_prefix: bpe
  model_type: BPE
  vocab_size: 5000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 9999
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
}
normalizer_spec {
  name: nmt_nfkc
  add_dummy_prefix: 1
  remove_extra_whitespaces: 1
  escape_whitespaces: 1
  normalization_r

['▁병신',
 '이',
 '▁아이',
 '스크',
 '림',
 '▁먹',
 '게',
 '▁돼',
 '▁있냐',
 '▁난',
 '▁먹으면',
 '▁안',
 '▁돼',
 '▁그만',
 '▁좀',
 '▁해',
 '▁당연히',
 '▁안',
 '▁되지',
 '▁너',
 '▁공부',
 '▁개',
 '못',
 '하는',
 '▁찐따',
 '잖아',
 '▁야',
 '▁',
 '닭',
 '대가',
 '리가',
 '▁말도',
 '▁하네',
 '▁동',
 '물',
 '농',
 '장',
 '▁나가',
 '자',
 '▁너네',
 '▁학교',
 '폭',
 '력',
 '으로',
 '▁신고하기',
 '▁전에',
 '▁그만',
 '▁해',
 '▁아이고',
 '▁무서워',
 '라',
 '▁살려주세요',
 '▁으',
 '헝헝',
 '▁제발',
 '▁좀',
 '▁저리',
 '▁가',
 '▁나',
 '였',
 '으면',
 '▁아이',
 '스크',
 '림',
 '▁먹을',
 '▁시간에',
 '▁공부',
 '한다',
 '▁멈춰',
 '▁저러',
 '니까',
 '▁쳐맞고',
 '▁다니지']

er.cc(258) LOG(INFO) Added: freq=1718 size=40 all=30644 active=4236 piece=▁못
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=1295 size=60 all=32056 active=5648 piece=렇게
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=964 size=80 all=33327 active=6919 piece=▁그냥
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=766 size=100 all=34393 active=7985 piece=▁얼
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=743 min_freq=47
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=661 size=120 all=35210 active=2521 piece=▁회
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=592 size=140 all=36099 active=3410 piece=▁어디
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=508 size=160 all=37041 active=4352 piece=▁회사
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=461 size=180 all=37965 active=5276 piece=▁넌
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=408 size=200 all=38595 active=5906 piece=▁돌
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=401 min_freq=39
bpe_model_tra

In [13]:
class_names = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화']

train_data = train_data[['class', 'conversation']]
train_data['class'] = train_data['class'].map(lambda c: class_names.index(c))
train_data = train_data.reset_index()

train_data.head()

_freq=17 min_freq=7
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=17 size=3320 all=75866 active=3910 piece=▁마시
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=17 size=3340 all=75926 active=3970 piece=▁옷을
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=17 size=3360 all=76023 active=4067 piece=버리면
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=17 size=3380 all=76107 active=4151 piece=▁몸으로
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=17 size=3400 all=76126 active=4170 piece=▁필요해
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=17 min_freq=7
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=17 size=3420 all=76144 active=3824 piece=▁아니지만
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=16 size=3440 all=76143 active=3823 piece=▁묶
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=16 size=3460 all=76311 active=3991 piece=자리
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=16 size=3480 all=76493 active=4173 piece=▁교통
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=16 size=3

,index,class,conversation
0,0,0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,0,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,3,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,1,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,1,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vocab_size = None

vector = TfidfVectorizer(preprocessor=preprocess_document, 
                         tokenizer=tokenizer.encode_as_pieces,
                         max_features=vocab_size)
print(vector.fit_transform(train_data['conversation']).toarray())
print("단어 개수:", len(vector.vocabulary_))

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.17453519 0.         0.         ... 0.         0.         0.        ]
 [0.19599194 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
단어 개수: 5239


In [15]:
x_data = vector.transform(train_data['conversation'])
y_data = train_data['class']

In [16]:
from sklearn.metrics import classification_report

def save_and_show_report(y_true, y_pred, prefix, show=True, zero_division=0):
    report = classification_report(y_true, y_pred, zero_division=zero_division)
    with open(BASE_DIR + '/' + prefix + '_report.txt', 'w') as f:
        f.write(report)
    if show:
        print(report)

In [17]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

def graph_confusion_matrix(model, x_test, y_test, classes_name, prefix, show=True):
    df_cm = pd.DataFrame(confusion_matrix(y_test, model.predict(x_test)), index=classes_name, columns=classes_name)
    fig = plt.figure(figsize=(12,12))
    heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=12)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=12)
    plt.ylabel('label')
    plt.xlabel('predicted value')
    plt.savefig(BASE_DIR + '/' + prefix + '_confusion.png')
    if show:
        plt.show()
    else:
        plt.close()

In [18]:
def train_and_eval(model, x_train, y_train, x_val, y_val, prefix):
    model.fit(x_train, y_train)
    
    save_and_show_report(y_train, model.predict(x_train), prefix + '_train', show=False)
    graph_confusion_matrix(model, x_train, y_train, class_names, prefix + '_train', show=False)
    
    save_and_show_report(y_val, model.predict(x_val), prefix + '_val', show=False)
    graph_confusion_matrix(model, x_val, y_val, class_names, prefix + '_val', show=False)
    
    return model

In [19]:
def create_submission(model, prefix):
    submission_data = []

    for file_name, row in test_data.iterrows():
        text = row['text']
        _input = vector.transform([text])
        predicted = model.predict(_input)[0]

        submission_data.append({
            'file_name': file_name,
            'class': predicted
        })
    
    submission_data = pd.DataFrame(submission_data)
    submission_data.to_csv(prefix + '_submission.csv', index=False)

In [21]:
from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score #정확도 계산

models = [
    (MultinomialNB, dict()),
    (ComplementNB, dict()),
    (LinearSVC, {'C': 1000, 'penalty': 'l1', 'max_iter': 50000, 'dual': False}),
    (DecisionTreeClassifier, {'max_depth': 20}),
    (RandomForestClassifier, {'n_estimators': 20, 'max_depth': 20}),
    (GradientBoostingClassifier, dict())
]

In [12]:
from sklearn.model_selection import KFold

kfold = KFold(shuffle=True)

In [13]:
import os
import time


#BASE_DIR = time.strftime("%m%d-%H%M%S", time.localtime())
BASE_DIR = 'sentencepiece'
os.mkdir(BASE_DIR)

for Model, options in models:
    for i, (train_indice, val_indice) in enumerate(kfold.split(x_data)):
        model = Model(**options)
        
        x_train = x_data[train_indice]
        y_train = y_data[train_indice]
        x_val = x_data[val_indice]
        y_val = y_data[val_indice]
        
        prefix = Model.__name__ + '_' + str(i)
        model = train_and_eval(model, x_train, y_train, x_val, y_val, prefix)
        
        print(f'{prefix}: {accuracy_score(y_train, model.predict(x_train)):.4f}, {accuracy_score(y_val, model.predict(x_val)):.4f}')

FileExistsError: [Errno 17] File exists: 'sentencepiece'

In [24]:
BASE_DIR = 'sentencepiece'

options = dict()
#options = {'n_estimators': 20, 'max_depth': 20}
model = ComplementNB(**options)
model = train_and_eval(model, x_data, y_data, x_data, y_data, 'CNB_final')
create_submission(model, 'sentencepiece_CNB')

In [ ]:
word_score_table.keys()

In [ ]:
test_corpus = test_data['text'].map(preprocess_document)

In [ ]:
test_seqs = vector.transform(test_corpus)
test_seqs = test_seqs.toarray()

In [ ]:
vocabulary = word_score_table.keys()

total_words = 0
coverage = 0

for row in test_seqs:
    total_words += len(row)
    for word in row:
        if word in vocabulary:
            coverage += 1
            
print('test coverage:', coverage / float(total_words))

In [ ]:
results = dict()

with open('SentencePiece.txt', 'r') as f:
    while True:
        line = f.readline()
        if line is None or line == '':
            break
        model_name, train_acc, val_acc = line.split()
        model_name = model_name[:-3]
        train_acc = float(train_acc[:-1])
        val_acc = float(val_acc[:-1])
        if results.get(model_name, None) is None:
            results[model_name] = {'train': 0.0, 'val': 0.0}
        results[model_name]['train'] += train_acc
        results[model_name]['val'] += val_acc

for model_name in results:
    print(model_name + ":")
    print(f"\ttrain: {results[model_name]['train'] / 5.0:.4f}")
    print(f"\tval: {results[model_name]['val'] / 5.0:.4f}")

In [ ]:
""